# Multiple Regression: Ordinary Least Squares (OLS) vs. Gradient Descent (GD)

Multiple regression is a statistical technique used to model the relationship between one dependent variable (or label) and two or more independent variables (or features). It is a powerful tool for understanding how changes in independent variables influence the dependent variable. While this modeling approach can be implemented using various strategies, I will focus on two main methods: 
- **Ordinary Least Squares (OLS)**, which derives the solution through pure mathematical analysis, 
- **Gradient Descent (GD)**, an iterative optimization algorithm often associated with machine learning.

Although Python packages exist for conducting multiple regression, this article will show how to derive the regression parameters using both OLS and GD from scratch. Then, it compare their performance and applicability based on the following criteria:
- **Computational Speed**: How quickly each method converges to a solution, particularly for datasets of varying sizes and dimensionalities.
- **Memory Usage**: The amount of memory required by each approach, especially for large datasets where matrix operations (OLS) may become prohibitive.
- **Convergence Behavior**: The robustness of GD to hyperparameter tuning (e.g., learning rate) and its ability to converge to an optimal solution in different conditions.
- **Numerical Stability**: How each method performs when faced with ill-conditioned data or high multicollinearity among independent variables.
- **Scalability**: Suitability for big data scenarios, where the efficiency of processing may differ significantly between the two methods.
- **Ease of Implementation**: Practical considerations, including code complexity, ease of debugging, and availability of libraries or tools.
- **Accuracy**: The ability of each method to minimize the error term (residuals) and produce reliable parameter estimates under different scenarios.

Finally, the article will explore scenarios where one method is preferable over the other. For example:
- OLS may excel in smaller datasets where computational resources are not constrained.
- GD is often the method of choice for very large datasets or when working with models that extend beyond linear regression, such as neural networks.

By analyzing these aspects, this article aims to provide a comprehensive comparison to help readers choose the appropriate method for their specific use case.

## A. Deriving Parameters

First, let's focus on how the parameters are derived with OLS and GD.

### How to Use OLS

#### 0) How to Get Started

We start with the following equation:

$$Xβ = y$$

Where:

*X* := The design matrix of independent variables\
*β* := The vector of parameters (coefficients to be estimated)\
*y* := The vector of the dependent variable

This is not possible, however, since there is always an error value, known as the *residuals*. 

Thus:

$$X\hat{β} + ε = \hat{y}$$

With that in mind, OLS can be derived through various approaches:

#### 1) **Optimization using calculus**  

In this scenario, we minimize the norm of $y - \hat{y}$, which will give us the vector of parameters. $$\min_β |y - \hat{y}| = \min_β |y - X\hat{β}|^2$$

I won't do the math for this, but if you're interested, it requires calculus and is a stronger solution since it follows along the maximum likelihood estimation (MLE).

#### 2) **Using the Left Inverse**

Because $Xβ$ is assumed to be full column rank, we can derive *β*.

In other words, if we multiply $Xβ$ by its transpose, we can then multiply it by its inverse.

Thus:

$$X^TXβ = X^Ty$$
$$(X^TX)^{-1}X^TXβ = (X^T X)^{-1}X^Ty$$
$$Iβ = (X^TX)^{-1}X^Ty$$
$$β = (X^T X)^{-1}X^Ty$$

Note:
*I* = The identity matrix

Because β is predicting y, it is better to conclude that this derivation represents $\hat{β}$.

This is not the best approach because we are assuming $Xβ$ is full column rank, which may not be true if there is **Multicollinearity**.

Why? If one parameter is a linear combination of any of the others, then the design matrix $X$ is not of full column rank (e.g. $β_3 = 4*β_1$).

#### 3) Transforming $X$ to Row Canonical Form

The equation

$$X^TXβ = X^Ty$$

can be row reduced to its canonical form to get the vector of parameters as well.

This will make sense in the next derivation.

#### **4) Using Orthogonal Projections**

$Xβ = y$ can be seen as having a projection and an orthogonal component.

If we view the parameters as predictions, then we have a projection:
$$X\hat{β} = \hat{y}$$

This is also known as $\hat{y}$ projected onto the column space of $X\hat{β}$

The orthogonal component will be the nullspace $X\hat{β}$, which is the residual: $y - \hat{y}$ (or $y - X\hat{β}$)

In other words, the *residual* (orthogonal) and *prediction* (projection) add up to the vector of the dependent variable: y

What now?

Given that $y - X\hat{β}$ is orthogonal to the column space of $X$, then it's orthogonal to each column of $X$:

$$X^T*(y - X\hat{β}) = 0$$
$$X^Ty - X^TX\hat{β} = 0$$
$$X^Ty = X^TX\hat{β}$$
$$(X^TX)^{-1}X^Ty = \hat{β}$$

We could have solved for that using row reduction, where $X^TX$ is augmented by $X^Ty$, then put into row canonical form.

#### **Conclusion**

Regardless of which approach you use with OLS, the coefficients will be:
$$\hat{β} = (X^TX)^{-1}X^Ty$$

### How to Use GD

<!-- #### **0) How to Get Started**

With GD, you must decide a cost function, AKA a loss or an objective function.

For multiple regression, we start with the mean square error (MSE) as the cost function (it is most common):

$$MSE = \frac{1}{N} \sum_{i=1}^N (y_i - \beta_0 - \mathbf{x}_i^T \boldsymbol{\beta})^2$$

Where:

*N* := Number of data points\
*y_i* := The vector of the dependent variable\
*β_0* := The scalar of the independent variable's intercept\
*β_i* := The vector of parameters (coefficients to be estimated)\
*x_i* := The vector of independent variables

Note: This has not been put in matrix form because GD uses calculus.

At first, you might think, "How can I calculate the residual without first having values for the parameters?"

GD iteratively determines the parameters' values.

You add the value of the partial derivative of the MSE with regard to the particular parameter:

$$\frac{\partial MSE}{\partial \beta_0} = -\frac{2}{N} \sum_{i=1}^N (y_i - \beta_0 - \mathbf{x}_i^T \boldsymbol{\beta_i})$$

$$\frac{\partial MSE}{\partial {\beta_i}} = -\frac{2}{N} \sum_{i=1}^{N}(y_i - \beta_0 + \mathbf{x}_i^T \boldsymbol{\beta_i}) * x_i$$

Now, let's set the parameters to 0 and add a learning rate (α) and number of times (i.e. epochs) to update each parameter.

$\alpha = 0.01$ will be the learning rate, and $t = 100$ will be the number of epochs.

We want to update the model parameters using the following equations:

$$\beta_0^{t+1} = \beta_0 + \alpha \cdot \frac{\partial MSE}{\partial \beta_0}$$

$$\beta_i^{t+1} = \beta_i + \alpha \cdot \frac{\partial MSE}{\partial \beta_i}$$

Substituting the derivatives we found earlier:

$$\beta_0^{t+1} = \beta_0^{t+1} + 0.01 \cdot (-\frac{2}{N} \sum_{i=1}^{N}(y_i - (\beta_0^{t+1} + \beta_i x_i)))$$

$$\beta_i^{t+1} = \beta_i + 0.01 \cdot (-\frac{2}{N} \sum_{i=1}^{N}(y_i - (\boldsymbol{\beta_0} + \beta_i \boldsymbol{x_i})) \cdot \boldsymbol{x_i})$$

Simplifying the equations:

$$\beta_0^{t+1} = \beta_0 - 0.02 \sum_{i=1}^{N}(y_i - (\boldsymbol{\beta_0} + \beta_i \boldsymbol{x_i}))$$

$$\beta_i^{t+1} = \beta_i - 0.02 \sum_{i=1}^{N}(y_i - (\boldsymbol{\beta_0} + \beta_i \boldsymbol{x_i})) \cdot \boldsymbol{x_i}$$ -->

#### **0) How to Get Started**

To perform Gradient Descent (GD), you must first decide on a cost function, also known as a loss or objective function.

For multiple regression, we typically use the mean squared error (MSE) as the cost function:

$$MSE = \frac{1}{N} \| \mathbf{y} - \beta_0 \mathbf{1} - \mathbf{X} \boldsymbol{\beta} \|^2$$

Where:

*N* := Number (scalar) of data points\
*y* := Vector of dependent variables (N * 1)\
*β_0* := Scalar intercept term\
*1* := Vector of 1's (n * 1)\
*X* := Matrix of independent variables (M * N)\
*β* := Vector of parameters (coefficients to be estimated, N * 1)

At first, you might think, "How can I calculate the residual without first having values for the parameters?"

GD iteratively determines the parameters' values updating the parameters at each step based on the gradient of the cost function, which, in this case, is the MSE.

You add the value of the partial derivative of the MSE with regard to the particular parameter:

$$\frac{\partial MSE}{\partial \beta_0} = -\frac{2}{N} \mathbf{1}^T (\mathbf{y} - \beta_0 \mathbf{1} - \mathbf{X} \boldsymbol{\beta})$$

$$\frac{\partial MSE}{\partial \boldsymbol{\beta}} = -\frac{2}{N} \mathbf{X}^T (\mathbf{y} - \beta_0 \mathbf{1} - \mathbf{X} \boldsymbol{\beta})$$

Now, let's set the parameters to 0 and add a learning rate (α) and number of times (i.e. epochs) to update each parameter.

$α = 0.01$ will be the learning rate, and $t = 100$ will be the number of epochs.

The updates for the parameters are:

$$\beta_0^{(t+1)} = \beta_0^{(t)} - \alpha \cdot \frac{\partial MSE}{\partial \beta_0}$$

$$\boldsymbol{\beta}^{(t+1)} = \boldsymbol{\beta}^{(t)} - \alpha \cdot \frac{\partial MSE}{\partial \boldsymbol{\beta}}$$

Substituting the derivatives into the update equations:

$$\beta_0^{(t+1)} = \beta_0^{(t)} + \frac{2 \alpha}{N} \mathbf{1}^T (\mathbf{y} - \beta_0^{(t)} \mathbf{1} - \mathbf{X} \boldsymbol{\beta}^{(t)})$$

$$\boldsymbol{\beta}^{(t+1)} = \boldsymbol{\beta}^{(t)} + \frac{2 \alpha}{N} \mathbf{X}^T (\mathbf{y} - \beta_0^{(t)} \mathbf{1} - \mathbf{X} \boldsymbol{\beta}^{(t)})$$

#### Conclusion

The expressions $\beta_0^{(t+1)}$ and $\boldsymbol{\beta}^{(t+1)}$ would give us the intercept and vector of parameters.

## B. Deriving Parameters Using OLS & GD

In [1]:
# The dataset will come from Kaggle:

import kagglehub

# Download latest version
path = kagglehub.dataset_download("nikhil7280/student-performance-multiple-linear-regression")
print("Path to dataset files:", path)

100%|██████████| 48.5k/48.5k [00:00<00:00, 21.3MB/s]

Extracting files...
Path to dataset files: /Users/alexdubro/.cache/kagglehub/datasets/nikhil7280/student-performance-multiple-linear-regression/versions/1


In [3]:
import matplotlib.pyplot as plt
import numpy as np 
import os
import pandas as pd 
import seaborn as sns 

dataset_path = os.path.join(path, 'Student_Performance.csv')

raw_data = pd.read_csv(dataset_path)
raw_data.head()

,Hours Studied,Previous Scores,Extracurricular Activities,Sleep Hours,Sample Question Papers Practiced,Performance Index
0,7,99,Yes,9,1,91.0
1,4,82,No,4,2,65.0
2,8,51,Yes,7,2,45.0
3,5,52,Yes,5,2,36.0
4,7,75,No,8,5,66.0


In [16]:
# 1) Removing null data

# print(raw_data.isnull().sum())
raw_data.dropna(inplace=True)

# 2) Removing duplicates

raw_data.drop_duplicates(inplace=True)
# duplicate_rows = raw_data[raw_data.duplicated()]
# print(duplicate_rows)

# Identify fully duplicate rows (entire row content matches)
# duplicate_rows = raw_data[raw_data.duplicated()]
# duplicate_rows.head()

# s = raw_data.duplicated()

# print(s)

# duplicateRows = raw_data[raw_data.duplicated(keep=False)]
# print(len(duplicateRows))

# Step 1: Identify all duplicate rows
# duplicate_rows = raw_data[raw_data.duplicated(keep=False)]

# Step 2: Display one example of a duplicate row
# example_duplicate = duplicate_rows.iloc[0] # Or use .iloc[0] for the first duplicate row
# print("Example of a duplicate row:\n", example_duplicate)


# Show duplicate rows
# print("Duplicate rows based on all columns:\n", duplicate_rows)

# duplicates = raw_data[raw_data.duplicated()]
# print("Duplicate rows:\n", duplicates)
# all_duplicates = raw_data[raw_data.duplicated(keep=False)]
# print(len(duplicate_rows))
 
# Print the resultant Dataframe
# print(duplicate)
# df.drop_duplicates(inplace=True)

We will be using the following equation to derive the parameters for OLS:

$$\hat{β} = (X^TX)^{-1}X^Ty$$

For multiple regression, X reflects the features; y is the label.

Features will include quantitative and categorical variables.

If there are categorical variables, they must be dummy-coded.

In [ ]:
raw_data['Extracurricular Activities'].value_counts()

In [ ]:
dums = pd.get_dummies(df['Extracurricular Activities'], dtype='float')
df['Extracurricular Activities'] = dums[['Yes']]

In [ ]:
df.head()

## C. Comparing OLS & GD